<a href="https://colab.research.google.com/github/ThinhVan27/BTL-NMAI-251/blob/bfs/BFS%20(Blind%20search).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, Iterator, List, Optional, Sequence, Tuple
from abc import ABC, abstractmethod
import time
import tracemalloc
import collections

Coord = Tuple[int, int]
Move = str
MOVE_VECTORS: Dict[Move, Tuple[int, int]] = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
}


class LevelFormatError(ValueError):
    """Raised when an ASCII level description cannot be parsed."""


@dataclass(frozen=True)
class SokobanBoard:
    width: int
    height: int
    walls: frozenset[Coord]
    goals: frozenset[Coord]
    initial_boxes: frozenset[Coord]
    initial_player: Coord

    @classmethod
    def from_ascii(cls, ascii_map: Sequence[str]) -> "SokobanBoard":
        if isinstance(ascii_map, str):
            raw_lines = ascii_map.splitlines()
        else:
            raw_lines = list(ascii_map)
        lines = [line.rstrip("\n") for line in raw_lines if line.strip()]
        if not lines:
            raise LevelFormatError("Level description is empty")
        height = len(lines)
        width = max(len(row) for row in lines)
        walls: set[Coord] = set()
        goals: set[Coord] = set()
        boxes: set[Coord] = set()
        player: Optional[Coord] = None
        for y, row in enumerate(lines):
            padded = row.ljust(width)
            for x, char in enumerate(padded):
                coord = (y, x) # Thay đổi tọa độ thành (hàng, cột) để khớp notebook gốc
                if char == "#":
                    walls.add(coord)
                elif char == ".":
                    goals.add(coord)
                elif char == "$":
                    boxes.add(coord)
                elif char == "@":
                    player = coord
                elif char == "*":
                    boxes.add(coord)
                    goals.add(coord)
                elif char == "+":
                    goals.add(coord)
                    player = coord
                elif char == "%":
                    goals.add(coord)

        if player is None:
            raise LevelFormatError("Player '@' or '+' is required")
        if not goals:
            raise LevelFormatError("At least one goal '.' is required")
        if not boxes:
            raise LevelFormatError("At least one box '$' or '*' is required")
        if len(boxes) != len(goals):
             print(f"Warning: Number of boxes ({len(boxes)}) does not match number of goals ({len(goals)}).")

        return cls(
            width=width,
            height=height,
            walls=frozenset(walls),
            goals=frozenset(goals),
            initial_boxes=frozenset(boxes),
            initial_player=player,
        )

    def is_wall(self, coord: Coord) -> bool:
        return coord in self.walls

    def is_goal(self, coord: Coord) -> bool:
        return coord in self.goals

    def in_bounds(self, coord: Coord) -> bool:
        y, x = coord
        return 0 <= y < self.height and 0 <= x < self.width

    def render(self, state: Optional["SokobanState"] = None) -> str:
        active_state = state or SokobanState(self.initial_player, self.initial_boxes)
        grid = [[" " for _ in range(self.width)] for _ in range(self.height)]
        for y, x in self.walls:
            grid[y][x] = "#"
        for y, x in self.goals:
            grid[y][x] = "."
        for y, x in active_state.boxes:
            grid[y][x] = "*" if (y, x) in self.goals else "$"
        py, px = active_state.player
        grid[py][px] = "+" if (py, px) in self.goals else "@"
        return "\n".join("".join(row) for row in grid)


@dataclass(frozen=True, order=True)
class SokobanState:
    player: Coord
    boxes: frozenset[Coord]

    def __init__(self, player: Coord, boxes: frozenset[Coord]):
        object.__setattr__(self, 'player', player)
        object.__setattr__(self, 'boxes', frozenset(sorted(list(boxes))))


class SokobanGame:
    def __init__(self, board: SokobanBoard):
        self._board = board
        self.initial_state = SokobanState(board.initial_player, board.initial_boxes)

    @property
    def board(self) -> SokobanBoard:
        return self._board

    def is_goal(self, state: SokobanState) -> bool:
        return state.boxes == self._board.goals
    def apply_move(self, state: SokobanState, move: Move) -> Optional[SokobanState]:
        if move not in MOVE_VECTORS:
            raise ValueError(f"Unknown move: {move}")
        dr, dc = MOVE_VECTORS[move]
        pr, pc = state.player
        nr, nc = pr + dr, pc + dc
        next_coord = (nr, nc)

        if self._board.is_wall(next_coord):
            return None

        boxes = set(state.boxes)
        if next_coord in boxes:
            br, bc = nr + dr, nc + dc
            box_target = (br, bc)
            if self._board.is_wall(box_target) or box_target in boxes:
                return None
            boxes.remove(next_coord)
            boxes.add(box_target)
            return SokobanState(player=next_coord, boxes=frozenset(boxes))

        return SokobanState(player=next_coord, boxes=state.boxes)

    def successors(self, state: SokobanState) -> Iterator[tuple[Move, SokobanState]]:
        for move in MOVE_VECTORS:
            next_state = self.apply_move(state, move)
            if next_state is not None:
                yield move, next_state

    def render(self, state: Optional[SokobanState] = None) -> str:
        return self._board.render(state or self.initial_state)

    def replay(self, moves: Iterable[Move]) -> List[SokobanState]:
        history = [self.initial_state]
        current = self.initial_state
        for move in moves:
            next_state = self.apply_move(current, move)
            if next_state is None:
                raise ValueError(f"Illegal move '{move}' encountered during replay")
            history.append(next_state)
            current = next_state
        return history


class ComplexityTracker:
    def __init__(self) -> None:
        self.nodes_expanded = 0
        self.max_frontier = 0
        self._start_time = 0.0
        self.peak_memory = 0

    def start(self) -> None:
        self._start_time = time.perf_counter()
        tracemalloc.start()

    def stop(self) -> float:
        elapsed = time.perf_counter() - self._start_time
        _, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory = peak
        return elapsed

    def record_node(self) -> None:
        self.nodes_expanded += 1

    def track_frontier(self, size: int) -> None:
        if size > self.max_frontier:
            self.max_frontier = size


@dataclass
class SolverLimits:
    time_limit_s: Optional[float] = None
    node_limit: Optional[int] = None


@dataclass
class SolverResult:
    solved: bool
    move_sequence: List[Move]
    elapsed_time_s: float
    nodes_expanded: int
    max_frontier: int
    peak_memory_bytes: int


class Solver(ABC):
    name: str = "base"
    def __init__(self) -> None:
        self.tracker = ComplexityTracker()

    def solve(self, game: SokobanGame, limits: Optional[SolverLimits] = None) -> SolverResult:
        limits = limits or SolverLimits()
        self.tracker = ComplexityTracker()
        self.tracker.start()
        solved = False
        moves = []
        try:
            solved, moves = self._search(game, limits)
        finally:
            elapsed = self.tracker.stop()
        return SolverResult(
            solved=solved,
            move_sequence=moves,
            elapsed_time_s=elapsed,
            nodes_expanded=self.tracker.nodes_expanded,
            max_frontier=self.tracker.max_frontier,
            peak_memory_bytes=self.tracker.peak_memory,
        )

    @abstractmethod
    def _search(self, game: SokobanGame, limits: SolverLimits) -> tuple[bool, List[Move]]:
        raise NotImplementedError


class SolutionVisualizer(ABC):
    @abstractmethod
    def display(self, game: SokobanGame, moves: List[Move]) -> None:
        raise NotImplementedError

class ConsoleVisualizer(SolutionVisualizer):
    def __init__(self, delay_s: float = 0.0) -> None:
        self.delay_s = delay_s

    def display(self, game: SokobanGame, moves: List[Move]) -> None:
        try:
            history = game.replay(moves)
            for index, state in enumerate(history):
                # Lệnh xóa màn hình cho console
                print("\033[H\033[J", end="")
                print(f"Step {index} / {len(moves)}")
                print(game.render(state))
                if self.delay_s:
                    time.sleep(self.delay_s)
        except ValueError as e:
            print(f"Error during replay: {e}")


In [ ]:
class BFSSolver(Solver):
    name = "bfs"

    def _search(self, game: SokobanGame, limits: SolverLimits) -> tuple[bool, List[Move]]:

        initial_state = game.initial_state

        # Hàng đợi (queue) chứa các tuple (trạng thái, danh_sách_nước_đi)
        queue = collections.deque([(initial_state, [])])

        # Tập hợp các trạng thái đã duyệt để tránh lặp
        visited_states = {initial_state}

        self.tracker.track_frontier(len(queue))

        while queue:
            # Lấy trạng thái hiện tại và đường đi ra khỏi hàng đợi
            current_state, current_moves = queue.popleft()
            self.tracker.record_node() # Ghi nhận đã mở rộng 1 node

            # Kiểm tra giới hạn thời gian và số node
            if limits.time_limit_s is not None and (time.perf_counter() - self.tracker._start_time) > limits.time_limit_s:
                print("Time limit exceeded.")
                return False, []
            if limits.node_limit is not None and self.tracker.nodes_expanded > limits.node_limit:
                print("Node limit exceeded.")
                return False, []

            # 2. Kiểm tra xem có phải trạng thái đích không
            if game.is_goal(current_state):
                return True, current_moves  # Trả về kết quả và chuỗi hành động

            # 3. Lấy các trạng thái kế tiếp hợp lệ
            for move, next_state in game.successors(current_state):
                # 4. Nếu trạng thái kế tiếp chưa được duyệt
                if next_state not in visited_states:
                    visited_states.add(next_state)
                    new_moves = current_moves + [move]
                    queue.append((next_state, new_moves))

            # Cập nhật kích thước lớn nhất của hàng đợi
            self.tracker.track_frontier(len(queue))

        # Nếu hàng đợi rỗng mà không tìm thấy giải pháp
        return False, []

In [ ]:
# 1. Định nghĩa màn chơi từ notebook gốc của bạn
maze_ascii = """
########
#@ $ % #
#  #   #
#  $ . #
########
"""

# 2. Tạo các đối tượng Game và Solver
# Tọa độ (hàng, cột) sẽ được đọc tự động từ chuỗi ở trên
board = SokobanBoard.from_ascii(maze_ascii)
game = SokobanGame(board)
solver = BFSSolver()

# 3. Chạy thuật toán giải
print("Attempting to solve Sokoban puzzle using the refactored BFS solver...")
result = solver.solve(game, limits=SolverLimits(time_limit_s=60)) # Đặt giới hạn 60s

# 4. In kết quả ra màn hình
print("\n--- Solver Result ---")
if result.solved:
    print("✅ Solution found!")
    print(f"   - Move sequence: {result.move_sequence}")
    print(f"   - Number of moves: {len(result.move_sequence)}")
else:
    print("❌ No solution found or limit exceeded.")

print("\n--- Performance ---")
print(f"   - Elapsed time: {result.elapsed_time_s:.4f} seconds")
print(f"   - Nodes expanded: {result.nodes_expanded}")
print(f"   - Max frontier size: {result.max_frontier}")
print(f"   - Peak memory usage: {result.peak_memory_bytes / 1024:.2f} KB")

Attempting to solve Sokoban puzzle using the refactored BFS solver...

--- Solver Result ---
✅ Solution found!
   - Move sequence: ['right', 'right', 'right', 'left', 'left', 'down', 'down', 'right', 'right']
   - Number of moves: 9

--- Performance ---
   - Elapsed time: 0.0094 seconds
   - Nodes expanded: 111
   - Max frontier size: 42
   - Peak memory usage: 66.01 KB


In [ ]:
if result.solved:
    print("\n--- Visualizing Solution ---")
    print("Press Ctrl+C to stop.")
    time.sleep(2) # Chờ 2 giây để người dùng đọc kết quả

    visualizer = ConsoleVisualizer(delay_s=0.5)
    try:
        visualizer.display(game, result.move_sequence)
        print("\nVisualization complete!")
    except KeyboardInterrupt:
        print("\nVisualization stopped by user.")


--- Visualizing Solution ---
Press Ctrl+C to stop.
Step 0 / 9
########
#@ $ . #
#  #   #
#  $ . #
########
Step 1 / 9
########
# @$ . #
#  #   #
#  $ . #
########
Step 2 / 9
########
#  @$. #
#  #   #
#  $ . #
########
Step 3 / 9
########
#   @* #
#  #   #
#  $ . #
########
Step 4 / 9
########
#  @ * #
#  #   #
#  $ . #
########
Step 5 / 9
########
# @  * #
#  #   #
#  $ . #
########
Step 6 / 9
########
#    * #
# @#   #
#  $ . #
########
Step 7 / 9
########
#    * #
#  #   #
# @$ . #
########
Step 8 / 9
########
#    * #
#  #   #
#  @$. #
########
Step 9 / 9
########
#    * #
#  #   #
#   @* #
########

Visualization complete!
